In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras.models import Model
import tensorflow.keras.optimizers as optimizers
import tensorflow.keras.metrics as metrics

import keras.backend as K

2022-12-06 21:42:53.845072: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
MAX_PLAY_LENGTH = 203

In [2]:
feature_arr = np.load("./seq_data/feature_arr_ids.npy")

In [8]:
features_to_keep = ['gameId', 'playId', 'frameId', 'nflId', 'team_indicator', 'adj_x', 'adj_y', 's', 'a', 'adj_o', 'adj_dir']

In [4]:
## Test to make sure data preprocessing was fine

In [5]:
feature_arr.shape

(8558, 203, 253)

In [6]:
example_time_step = feature_arr[0][-1]

assert example_time_step[0] > 2000  # should be a game id
assert example_time_step[4] in [0,1,2]  # should be team indicator
assert example_time_step[0] == example_time_step[0 + len(features_to_keep)]  # should be a game id
assert example_time_step[4 + len(features_to_keep)] in [0,1,2]  # should be team indicator

In [7]:
example_state_space = feature_arr[0][-1].reshape(-1, len(features_to_keep))
print(example_state_space.shape)
print(example_state_space[0:3])

# looks good

(23, 11)
[[ 2.0210909e+09  9.7000000e+01  4.0000000e+01  2.5511000e+04
   1.0000000e+00  8.7310000e+01  6.1500000e+00  3.8300000e+00
   9.4000000e-01  2.7900000e+00  1.1255000e+02]
 [ 2.0210909e+09  9.7000000e+01  4.0000000e+01  3.5481000e+04
   1.0000000e+00  7.3120000e+01 -1.8020000e+01  6.2500000e+00
   2.0900000e+00  1.5800000e+00  2.9624000e+02]
 [ 2.0210909e+09  9.7000000e+01  4.0000000e+01  3.5634000e+04
   1.0000000e+00  6.5280000e+01 -1.4540000e+01  3.9500000e+00
   2.9500000e+00  1.2110000e+01  6.2990000e+01]]


In [8]:
train_index_end = 6000
val_index_end = 7300

x_train = feature_arr[0:train_index_end, :, :]
x_val = feature_arr[train_index_end:val_index_end, :, :]
x_test = feature_arr[val_index_end:, :, :]

x_train_shape = x_train.shape
x_val_shape = x_val.shape
x_test_shape = x_test.shape

# looks good

In [9]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)



(6000, 203, 253)
(1300, 203, 253)
(1258, 203, 253)


In [10]:
x_val[0][-1].reshape(-1, len(features_to_keep))[0:3,:]
# val looks good

array([[2.0211017e+09, 2.3270000e+03, 2.8000000e+01, 4.3290000e+04,
        1.0000000e+00, 8.5840000e+01, 1.8900000e+00, 6.8000000e-01,
        1.5400000e+00, 1.5770000e+01, 2.5118000e+02],
       [2.0211017e+09, 2.3270000e+03, 2.8000000e+01, 4.3299000e+04,
        2.0000000e+00, 6.8830000e+01, 1.7170000e+01, 5.2500000e+00,
        4.6100000e+00, 2.7461000e+02, 3.0631000e+02],
       [2.0211017e+09, 2.3270000e+03, 2.8000000e+01, 4.3350000e+04,
        2.0000000e+00, 6.8740000e+01, 4.3600000e+00, 2.5200000e+00,
        7.4000000e-01, 2.1797000e+02, 2.7068000e+02]])

In [11]:
train_mu = np.mean(x_train.reshape(-1,x_train.shape[-1]),axis=0)
train_std = np.std(x_train.reshape(-1,x_train.shape[-1]),axis=0)
train_std[train_std == 0] = 1

In [12]:
print(train_mu.shape)
print(train_std.shape)

(253,)
(253,)


In [14]:
# This normalization didn't work - didn't preserve things correctly

# x_train[:,:,5:] = ((x_train.reshape(-1,x_train.shape[-1])[:,5:] - train_mu[5:])/train_std[5:]).reshape((x_train_shape[0], x_train_shape[1], -1))
# x_val[:,:,5:] = ((x_val.reshape(-1,x_val.shape[-1])[:,5:] - train_mu[5:])/train_std[5:]).reshape((x_val_shape[0], x_val_shape[1], -1))
# x_test[:,:,5:] = ((x_test.reshape(-1,x_test.shape[-1])[:,5:] - train_mu[5:])/train_std[5:]).reshape((x_test_shape[0], x_test_shape[1], -1))

In [16]:
x_test = np.load("./seq_unnorm_data/x_train.npy")

In [17]:
x_test.shape

(6000, 203, 253)

In [18]:
x_test[0][-1].reshape(-1, len(features_to_keep))[0:3,:]

array([[ 2.0211014e+09,  4.1900000e+02,  3.2000000e+01,  3.5441000e+04,
         2.0000000e+00,  7.9280000e+01, -1.0300000e+00,  1.5100000e+00,
         1.9000000e+00,  1.9496000e+02,  1.9643000e+02],
       [ 2.0211014e+09,  4.1900000e+02,  3.2000000e+01,  3.5454000e+04,
         2.0000000e+00,  8.2670000e+01,  2.0300000e+00,  2.0600000e+00,
         4.8000000e+00,  1.1637000e+02,  1.3599000e+02],
       [ 2.0211014e+09,  4.1900000e+02,  3.2000000e+01,  3.7229000e+04,
         2.0000000e+00,  6.5510000e+01,  2.2470000e+01,  7.1000000e+00,
         3.0500000e+00,  1.5810000e+01,  1.9920000e+01]])

In [22]:
x_test.reshape(-1, MAX_PLAY_LENGTH, 23, 11)[:,:,:,4:].reshape(-1,MAX_PLAY_LENGTH,23*7)[0][-1].reshape(-1, 7)[0:3,:]

array([[  2.  ,  79.28,  -1.03,   1.51,   1.9 , 194.96, 196.43],
       [  2.  ,  82.67,   2.03,   2.06,   4.8 , 116.37, 135.99],
       [  2.  ,  65.51,  22.47,   7.1 ,   3.05,  15.81,  19.92]])

In [15]:
import copy

In [14]:
x_train.reshape(-1,x_train_shape[-1]).reshape(x_train_shape)[0][-1].reshape(-1, len(features_to_keep))[0:3,:]

array([[ 2.0210909e+09,  9.7000000e+01,  4.0000000e+01,  2.5511000e+04,
         1.0000000e+00,  8.7310000e+01,  6.1500000e+00,  3.8300000e+00,
         9.4000000e-01,  2.7900000e+00,  1.1255000e+02],
       [ 2.0210909e+09,  9.7000000e+01,  4.0000000e+01,  3.5481000e+04,
         1.0000000e+00,  7.3120000e+01, -1.8020000e+01,  6.2500000e+00,
         2.0900000e+00,  1.5800000e+00,  2.9624000e+02],
       [ 2.0210909e+09,  9.7000000e+01,  4.0000000e+01,  3.5634000e+04,
         1.0000000e+00,  6.5280000e+01, -1.4540000e+01,  3.9500000e+00,
         2.9500000e+00,  1.2110000e+01,  6.2990000e+01]])

In [ ]:
cop = copy.deepcopy(x_train.reshape(-1,x_train_shape[-1]))

cop[:]

In [50]:
x_train.flatten().reshape(x_train_shape)[0][-1].reshape(-1, len(features_to_keep))[0:3,:]

: 

: 

In [49]:
x_val.transpose(2,0,1).reshape(-1,x_val_shape[-1])[-1].reshape(-1, len(features_to_keep))[0:3,:]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [4]:
x_val.shape

(1300, 203, 253)

In [6]:
x_val[0][-1][0:3]

array([2.0211017e+09, 2.3270000e+03, 2.8000000e+01])

In [7]:
x_val[0][-1][4:11]

array([ 1.        ,  3.08189874,  0.62953999,  0.25165232,  1.54218847,
       -0.1560783 ,  2.9113318 ])

In [8]:
x_val[0][-1][12:15]

array([2.11556175, 2.22141293, 2.59839161])

In [11]:
x_val[0][-1].reshape(23,-1)

array([[ 2.02110170e+09,  2.32700000e+03,  2.80000000e+01,
         4.32900000e+04,  1.00000000e+00,  3.08189874e+00,
         6.29539994e-01,  2.51652325e-01,  1.54218847e+00,
        -1.56078303e-01,  2.91133180e+00],
       [ 2.27122355e+00,  2.11556175e+00,  2.22141293e+00,
         2.59839161e+00,  2.90329814e+00,  2.43292136e+00,
         5.21522985e+00,  3.75868070e+00,  4.90316604e+00,
         3.11618021e+00,  3.51657135e+00],
       [ 2.27122355e+00,  2.11556175e+00,  2.22141293e+00,
         2.49403962e+00,  3.00990694e+00,  2.41502604e+00,
         1.35101790e+00,  1.65503874e+00,  4.99064482e-01,
         2.41492950e+00,  3.10048793e+00],
       [ 2.27122355e+00,  2.11556175e+00,  2.22141293e+00,
         2.42682690e+00,  1.32100520e+00,  3.06011713e+00,
         1.32380887e+00,  1.90062185e+00,  2.28180766e+00,
         4.00283831e+00,  1.94254024e+00],
       [ 2.27122355e+00,  2.11556175e+00,  2.22141293e+00,
         2.37355049e+00,  3.00450381e+00,  3.03113621e+00,
  

In [ ]:
x_val.reshape(-1, MAX_PLAY_LENGTH, 23, 11)[:,:,:,4:].reshape(-1,MAX_PLAY_LENGTH,23*7)